# Best Video Game Analysis

### Introduction: In this project, we will be trying to determine which is the best video game ever. (Released on steam)

In this project, we aim to determine which is the best video game released on Steam. Since it would be impossible to analyze every game ever created, we will focus on Steam, one of the largest digital platforms for PC games. The dataset we are using contains games going back to 1997 and includes around 200,000 entries — a sufficiently large sample for meaningful analysis.

Our goal is to rank these games based on statistics, taking into account factors like playtime, reviews, achievements, and ownership.

### Data preperation: 
So, I was initially going to use 2-3 datasets and merge them, but after doing some research I realized that there almost no - good, up to date, clean, meaningful datasets about video games.
I found one which is not bad but it's still filled with issues and missing information. Before I start modeling it I will try to create my own dataset using the Steam API and Steam Spy API to extract all available information about all videogames and then merging them, this process takes a long time so I will let the script run for a night, if it fails because of rate limits, memory limitation or some other issue I will go back to the Kaggle dataset and start preparing it. If it does work tho, I will simply continue work on my own dataset.

- Steam API has the official MetaData (release data, price, developers)
- Steam Spy has estimated/publically scraped data (owners, playtime, achivements)

In [2]:
import pandas as pd
import requests

In [24]:
pd.set_option('display.max_columns', None)

In [ ]:
games = pd.read_csv("../data/games.csv")
games = games.loc[
    games.groupby('Name')['Median playtime forever'].idxmax()
]

#games[games.Name.str.contains("Tom Clancy's Rainbow Six® Siege") & games.Name.notna()]
games

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,test,DiscountDLC count,About the game,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
81427,2556940,! Shakabula *,"Oct 13, 2023",0 - 20000,0,0,14.99,0,0,Be prepared to take on a terrifying South Afri...,...,0,0,0,Skermunkel,Skermunkel,"Single-player,Full controller support,Steam Cloud","Action,Indie,RPG,Early Access","Early Access,Action,RPG,JRPG,Shooter,Bullet He...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
31138,449940,! That Bastard Is Trying To Steal Our Gold !,"Mar 3, 2016",20000 - 50000,1,0,2.99,0,0,It's a fun puzzle game where you need to steal...,...,0,388,0,WTFOMGames,WTFOMGames,"Single-player,Steam Trading Cards,Partial Cont...","Action,Adventure,Casual,Indie","Puzzle-Platformer,Dark Humor,Dungeon Crawler,M...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
35088,1287250,! Wild Russia !,"Apr 28, 2020",0 - 20000,1,0,19.99,0,0,«Wild Russia» is an open-world horror adventur...,...,0,0,0,Andreev Worlds,Andreev Worlds,"Single-player,Steam Achievements,Partial Contr...","Action,Adventure,Casual","Adventure,Action,Casual,Horror,Post-apocalypti...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
18961,866510,!AnyWay!,"Jun 6, 2018",50000 - 100000,13,0,1.79,0,37,Let's Return to Our Past! An unusual world for...,...,0,359,0,EYEFRONT,EYEFRONT,"Single-player,Multi-player,Steam Achievements,...","Adventure,Casual,Indie","Side Scroller,Precision Platformer,2D Platform...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
51433,870990,!LABrpgUP!,"Jun 13, 2018",0 - 20000,0,0,0.99,0,0,LabRpgUp is a unique J-RPG game. Travel throug...,...,0,0,0,UPandQ,UPandQ,Single-player,"Adventure,Casual,Indie,RPG","Adventure,Indie,Casual,RPG",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33527,460250,🔴 Circles,"Feb 17, 2017",0 - 20000,0,0,2.99,0,0,"Circles is a unique, intuitive puzzle game whe...",...,0,0,0,Jeroen Wimmers,Jeroen Wimmers,"Single-player,Steam Achievements,Steam Cloud","Casual,Indie","Experimental,Minimalist,Relaxing,Abstract,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
42364,965340,🚀 Human Rocket Person,"Nov 14, 2018",0 - 20000,0,0,1.99,0,0,Join Human Rocket Person on the adventure of a...,...,0,0,0,2nd Studio,2nd Studio,"Single-player,Steam Achievements,Full controll...","Action,Indie,Simulation","Indie,Action,Sexual Content,Nudity,Simulation,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
52519,806220,🚀👾Absolute Blue 👾🚀,"Apr 21, 2018",0 - 20000,0,0,3.99,0,1,Note: I am currently working on a new ambitiou...,...,0,0,0,Intermediaware,Intermediaware,"Single-player,Partial Controller Support","Action,Indie","Action,Indie,Shoot 'Em Up,Bullet Hell,Arcade,S...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
10135,576960,🧠 OUT OF THE BOX,"Jul 19, 2018",20000 - 50000,1,17,14.99,0,0,Welcome to the Craziest Nightclub In this boun...,...,0,326,0,Nuclear Tales,🚀 Raiser Games,"Single-player,Steam Achievements,Steam Trading...","Adventure,Casual,Indie,RPG,Simulation,Strategy","Adventure,Choose Your Own Adventure,Simulation...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
